In [108]:
#必要なライブラリのインポート
import pandas as pd
import numpy as np

In [109]:
#データのインポート
teams=pd.read_csv("data/teams.csv")
boxscore=pd.read_csv("data/games_boxscore_202021.csv")

In [110]:
#teamsからB1_202021シーズンのTeamIdを取り出す。
#この作業はboxscoreからB1の選手のデータだけを取り出すために必要。
teams=teams[teams["Season"]=="2020-21"]
teams=teams[teams["League"]=="B1"]

In [111]:
#teamsとboxscoreをTeamIdでmergeして、B1の選手だけを抜き出す。
boxscore=teams.merge(boxscore,on="TeamId")

In [112]:
#選手のシーズン平均のスタッツを作るために、Player（選手名）でgroupbyし、mean()メソッドを使う。
#あとで同様の手順で標準偏差を計算するので、boxscoreに上書きしないように注意。
avg=boxscore.groupby("Player").mean()

In [113]:
#選手のスタッツの標準偏差を計算するために、Playerでgroupbyし、std()メソッドを使う。
std=boxscore.groupby("Player").std()

In [114]:
#avgとstdのインデックスをリセットする。リセットしないと次のブロックの処理ができない。
avg=avg.reset_index()
std=std.reset_index()

In [115]:
#DataFrameのlocメソッドを使って、avgとstdから必要な列だけ抜き出して上書きする。
avg=avg.loc[:,["Player","PTS","MIN"]]
std=std.loc[:,["Player","PTS"]]

In [116]:
#avgとstdをPlayerでmergeする。
#第三引数のsuffixesはavgとstdでPTSという同じ列名が存在するので、区別するためにつける目印を指定している。
data=avg.merge(std,on="Player",suffixes=("_avg","_std"))

In [117]:
data

,Player,PTS_avg,MIN,PTS_std
0,LJ・ピーク,11.396552,20.099655,5.606824
1,アイザイア・マーフィー,9.527273,24.027273,5.836839
2,アイラ・ブラウン,14.092593,31.810185,6.411277
3,アキ・チェンバース,9.384615,25.136923,5.885022
4,アレックス・カーク,17.085714,28.848857,6.594880
...,...,...,...,...
297,高原晟也,1.714286,4.714286,3.147183
298,高島紳司,4.363636,15.682727,5.025390
299,高橋耕陽,3.363636,8.992000,4.274661
300,鵤誠司,5.241379,21.414828,3.460607


In [118]:
#変動係数(CV)を計算する
#一試合平均得点が0の場合、欠損値Nanをリストに追加するようにif文を書く。0で割るとエラーが出るので注意。
CV=[]
for average,standard_deviation in zip(data["PTS_avg"],data["PTS_std"]):
    if average==0:
        CV.append(np.nan)
    else:
        CV.append(standard_deviation/average)
    
data["CV"]=CV

In [119]:
#平均出場時間が20分以上の選手で、変動係数が高い選手TOP10を表示する。
#sort_valuesメソッドの引数byでCVを基準に並べ替えを指示。ascendingは昇順or降順の指定で、Falseにすると降順になる。
#headメソッドで上から何行を表示するか決めている。たとえば、50に変えるとTOP50のランキングになる。
data[data["MIN"]>=20].sort_values(by="CV",ascending=False).head(10)

,Player,PTS_avg,MIN,PTS_std,CV
99,三ツ井利也,2.937500,22.690417,3.198445,1.088832
213,永吉佑也,3.017544,21.067544,2.979041,0.987240
199,栗原ルイス,5.659091,21.056136,5.198136,0.918546
153,太田敦也,4.406780,20.015763,3.644167,0.826946
195,林翔太郎,5.259259,22.445741,4.348867,0.826897
78,ファイパプ月瑠,3.770833,20.428125,3.033428,0.804445
132,北川弘,5.250000,21.687500,4.172651,0.794791
179,川村卓也,6.981132,21.589623,5.149647,0.737652
74,ハビエル・カーター,8.912281,21.973509,6.525447,0.732186
134,古川孝敏,9.118644,21.765254,6.568170,0.720301


In [120]:
#平均得点が10点以上で、変動係数が高い選手TOP10を表示する。
data[data["PTS_avg"]>=10].sort_values(by="CV",ascending=False).head(10)

,Player,PTS_avg,MIN,PTS_std,CV
91,リチャード・ヘンドリックス,12.200000,30.943000,7.997222,0.655510
81,ペリン・ビュフォード,13.541667,23.858542,8.346125,0.616329
93,ルーク・エヴァンス,10.500000,24.697500,6.454972,0.614759
9,アンドリュー・ランダル,10.250000,22.477500,6.020797,0.587395
90,リチャード・ソロモン,14.357143,21.599524,8.360247,0.582306
51,ジョナサン・オクテウス,12.527273,18.294182,7.231245,0.577240
11,ウィリアムスニカ,10.766667,22.555833,6.112967,0.567768
44,ジャワッド・ウィリアムズ,10.040816,24.433061,5.653020,0.563004
282,金丸晃輔,16.163636,26.251818,9.097748,0.562853
52,ジョーダン・テイラー,13.673469,22.633469,7.597663,0.555650


In [121]:
#出場時間が20分以上で、変動係数が低い選手TOP10を表示する。
data[data["MIN"]>=20].sort_values(by="CV",ascending=True).head(10)

,Player,PTS_avg,MIN,PTS_std,CV
12,ウィル・クリークモア,17.250000,32.240000,3.593976,0.208346
87,ライアン・ケリー,21.061224,28.415102,6.371840,0.302539
57,ステヴァン・イェロヴァツ,18.846154,30.863077,5.902424,0.313190
59,セバスチャン・サイズ,17.087719,27.998421,5.568665,0.325887
72,ニック・メイヨ,21.490909,33.622727,7.062873,0.328645
41,ジャック・クーリー,16.363636,28.086364,5.379535,0.328749
94,レイヴォンテ・ライス,19.980392,25.609608,6.595423,0.330095
35,ジェイソン・ウォッシュバーン,17.021739,29.255217,5.670544,0.333135
26,グレゴリー・エチェニケ,17.090909,32.474909,5.988485,0.350390
64,ディージェイ・ニュービル,19.340000,30.405800,6.793184,0.351250


In [122]:
#平均得点が10点以上で、変動係数が低い選手TOP10を表示する。
#先程のランキングでは得点が全員10点以上だったため、ランキングは全く同じになる。
data[data["PTS_avg"]>=10].sort_values(by="CV",ascending=True).head(10)

,Player,PTS_avg,MIN,PTS_std,CV
12,ウィル・クリークモア,17.250000,32.240000,3.593976,0.208346
87,ライアン・ケリー,21.061224,28.415102,6.371840,0.302539
57,ステヴァン・イェロヴァツ,18.846154,30.863077,5.902424,0.313190
59,セバスチャン・サイズ,17.087719,27.998421,5.568665,0.325887
72,ニック・メイヨ,21.490909,33.622727,7.062873,0.328645
41,ジャック・クーリー,16.363636,28.086364,5.379535,0.328749
94,レイヴォンテ・ライス,19.980392,25.609608,6.595423,0.330095
35,ジェイソン・ウォッシュバーン,17.021739,29.255217,5.670544,0.333135
26,グレゴリー・エチェニケ,17.090909,32.474909,5.988485,0.350390
64,ディージェイ・ニュービル,19.340000,30.405800,6.793184,0.351250
